In [34]:
# convert SemEval data to the standard format

from glob import glob 
import codecs
from xml.etree import ElementTree as et
from traceback import format_exc
from collections import defaultdict


contexts_fpaths = "/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/*.xml"
keys_fpath = "/Users/alex/work/joint/eval/semeval-2013-13/keys/gold/all.key"
output_fpath = "/Users/alex/Desktop/output-semeval-2013.csv"

# get keys
with codecs.open(keys_fpath, "r", "utf-8") as keys:
    context_id2sense_ids = {}
    for line in keys:
        try:
            fields = line.split()
            target = fields[0]
            context_id = fields[1]
            golden_sense_ids = fields[2:]
            context_id2sense_ids[context_id] = golden_sense_ids
        except:
            print "bad line: '%s'" % line.strip()
            print format_exc()
            

# parse xml
# "<instance id="appear.v.1" lemma="appear" partOfSpeech="v" token="appear" tokenEnd="65" tokenStart="59">Tone it down a tad, or at least bring a froth cup when you appear        before cameras.)</instance>"
with codecs.open(output_fpath, "w", "utf-8") as out:
    print >> out, "context_id\ttarget\ttarget_position\tgold_sense_ids\tpredict_sense_ids\tgolden_related\tpredict_related\tcontext"

    for word_fpath in glob(contexts_fpaths):
        print word_fpath

        tree = et.parse(word_fpath)
        root = tree.getroot()

        for child in root:
            if child.tag == "instance":
                print >> out, "%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % (
                                          child.attrib["id"],
                                          child.attrib["lemma"] + "#" + child.attrib["partOfSpeech"],
                                          child.attrib["tokenStart"]+  "," + child.attrib["tokenEnd"], 
                                          ",".join(context_id2sense_ids[child.attrib["id"]]),
                                          "",
                                          "",
                                          "",
                                          child.text)

print output_fpath
        

/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/add.v.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/appear.v.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/ask.v.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/become.v.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/board.n.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/book.n.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/book.v.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/color.n.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/common.j.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/control.n.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/dark.j.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/date.n.xml
/Users/alex/work/joint/eval/semeval-2013-13/contexts/xml-format/dismiss.v.xml
/Users/alex/w